<a href="https://colab.research.google.com/github/Sheebahlangat/Data-scince-and-AI-projects/blob/main/Generative_Ai_and_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import Libraries
!pip install -U langchain-community

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline

In [4]:
!pip install -U langchain-community

In [11]:
# --- Load PDF ---
!pip install pypdf
loader = PyPDFLoader("lekl101.pdf")
docs = loader.load()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 1.5 MB/s eta 0:00:00


In [12]:
# --- Split Documents into Chunks ---
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)


In [16]:
# --- Create Embeddings and Vector Store ---
!pip install faiss-cpu
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def query_rag(question):
    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    prompt = f"Answer the question using only the context:\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    response = flan_pipeline(
        prompt,
        max_new_tokens=200,
        temperature=0.9,      # Creativity control (lower = deterministic, higher = more diverse)
        top_k=50,             # Only sample from the top-k most likely tokens
        top_p=0.9,            # Nucleus sampling: only sample from tokens with cumulative prob <= top_p
        do_sample=True        # Enables sampling (required for temperature/top-k/top-p to work)
    )
    return response[0]['generated_text']



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Device set to use cpu


In [17]:
print(query_rag("Summarize the key points of this document in a paragraph of 200 words."))

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Suggested Reading One Hundred Years in Solitude by Gabriel Gar cia Marquez Love in the T ime of Cholera by Gabriel Gar cia Marquez. Reprint 2025-26 prose fiction, right from the really short ‘short story’ of about five hundred words to longer and more complex works. The longer ones, with their status of middle length, fall between the tautness of the short narrative and the expansiveness of the novel. There can be thematic variation too. The stories deal with fantasy, reality, alienation and the problem of choice in personal life. There are three short stories and two long ones in this section representing writers from five cultures. under a bright sun, a huge wave picked up several cars that were driving down the avenue along the seawall or parked on the pavement, and embedded one of them in the side of the hotel.
